<a href="https://colab.research.google.com/github/caltdreamer/GNN/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q torch-geometric
!pip install ogb
!pip install umap-learn
!pip install conditionalconformal
!git clone https://github.com/msesia/arc.git

In [4]:
import torch
import math
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GraphSAGE
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import NELL
from torch_geometric.datasets import CitationFull
from torch_geometric.datasets import CoraFull
from torch_geometric.datasets import Coauthor
from torch_geometric.data import DataLoader
from torch_geometric.nn import BatchNorm
from conditionalconformal.condconf import setup_cvx_problem_calib
import matplotlib.pyplot as plt
from tqdm import tqdm
import cvxpy as cp
import arc
from arc import models
from arc import methods
from arc import black_boxes
from arc import others
from arc import coverage
import torch.optim as optim
import numpy as np
import os.path as osp
import pandas as pd
import numpy as np
import collections
import random
import networkx as nx
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import arc
from arc import models
from arc import methods
from arc import black_boxes
from arc import others
from arc import coverage

from sklearn.neighbors import NearestNeighbors
from conditionalconformal.synthetic_data import generate_cqr_data, indicator_matrix
from conditionalconformal import CondConf

from scipy.stats import norm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from tqdm import tqdm

In [10]:
#dataset = CoraFull(root='./data')
dataset = Planetoid(root='./data', name='Cora',split = 'public')
#dataset = Coauthor(root = './data',name ='Physics' )
data = dataset[0]
original_x = data.x
for data in dataset:
    data.x = data.x / np.linalg.norm(data.x, axis=1, keepdims=True)

# Now, the feature vectors in the 'dataset' have been normalized with respect L2 norm.

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = data.x.to(device)
y = data.y.squeeze().to(device)


In [ ]:
print(data.x)

In [ ]:
#Randomly choose train and calitest set
marginal_condi = []
marginal_split = []
prediction_split = []
prediction_condi=[]
test_index=[]
train_index=[]
calib_index=[]
train_mask_list=[]
calib_mask_list=[]
test_mask_list=[]

train_percentage = 0.3
test_percentage = 0.5

for c in range(5):

    num_nodes = data.num_nodes
    all_node_indices = list(range(num_nodes))
    random.shuffle(all_node_indices)
    num_train_nodes = int(train_percentage * num_nodes)
    num_testandcali_nodes = num_nodes - num_train_nodes
    train_node_indices = all_node_indices[:num_train_nodes]
    testandcali_node_indices = all_node_indices[num_train_nodes:]

    #print(testandcali_node_indices)
    test_node_indices = testandcali_node_indices[:int(test_percentage * num_testandcali_nodes)]
    cali_node_indices = testandcali_node_indices[int(test_percentage * num_testandcali_nodes):]

    #print(test_node_indices)

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    train_mask[train_node_indices] = True
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    train_mask[train_node_indices] = True

    testandcali_mask = torch.zeros(num_nodes,dtype=torch.bool)
    testandcali_mask[testandcali_node_indices] = True

    test_mask = torch.zeros(num_nodes,dtype=torch.bool)
    test_mask[test_node_indices] = True


    cali_mask = torch.zeros(num_nodes,dtype=torch.bool)
    cali_mask[cali_node_indices] = True
    #***************************************************
    train_index.append(train_mask)
    calib_index.append(cali_mask)
    test_index.append(test_mask)
    #***************************************************
    train_loader = NeighborSampler(
      data.edge_index, node_idx=train_mask,
      sizes=[2,2,2], batch_size=32, shuffle=False,
    )
    #***************************************************
    class SAGE(torch.nn.Module):
      def __init__(self,in_channels,hidden_channels,out_channels,num_layers = 3):
          super(SAGE,self).__init__()

          self.numlayers = num_layers
          self.convs = torch.nn.ModuleList()
          self.convs.append(SAGEConv(in_channels,hidden_channels))
          for i in range(num_layers - 2):
              self.convs.append(SAGEConv(hidden_channels,hidden_channels))
          self.convs.append(SAGEConv(hidden_channels,out_channels))
      def reset_parameters(self):
          for conv in self.convs:
              conv.reset_parameters()
      def forward(self,x,adjs):
          for i ,(edge_index,_,size) in enumerate(adjs):
              xs = []
              x_target = x[:size[1]]#x should be a list prioritizing target nodes
              x = self.convs[i]((x,x_target),edge_index)
              if i != self.numlayers -1:
                  x = F.relu(x)
                  x = F.dropout(x,p=0.5,training=self.training)
              xs.append(x)
              if i == 0:
                  x_all = torch.cat(xs, dim=0)
                  layer_1_embeddings = x_all
              elif i == 1:
                  x_all = torch.cat(xs, dim=0)
                  layer_2_embeddings = x_all
              elif i == 2:
                  x_all = torch.cat(xs, dim=0)
                  layer_3_embeddings = x_all
          return layer_1_embeddings, layer_2_embeddings, layer_3_embeddings
    """(In situation when we aim to use all neighborhood information)
      def inference(self, x,adjs):
          for i ,(edge_index,_,size) in enumerate(adjs):
              xs = []
              x_target = x[:size[1]]#x should be a list prioritizing target nodes
              x = self.convs[i]((x,x_target),edge_index)
              if i != self.numlayers -1:
                  x = F.relu(x)
                  x = F.dropout(x,p=0.5,training=self.training)
              xs.append(x)
              if i == 0:
                  x_all = torch.cat(xs, dim=0)
                  layer_1_embeddings = x_all
              elif i == 1:
                  x_all = torch.cat(xs, dim=0)
                  layer_2_embeddings = x_all
              elif i == 2:
                  x_all = torch.cat(xs, dim=0)
                  layer_3_embeddings = x_all
          return layer_1_embeddings, layer_2_embeddings, layer_3_embeddings
    """
    #***************************************************
    model = SAGE(dataset.num_features, 128, dataset.num_classes, num_layers=3)

    model.reset_parameters()
    #We only input the initializing variables
    model = model.to(device)
    #***************************************************
    def train(epoch):
      model.train()
      total_loss = total_correct =0

      for batch_size,n_id,adjs in train_loader:
          adjs = [adj.to(device) for adj in adjs]
          optimizer.zero_grad()
          l1_emb, l2_emb, l3_emb = model(x[n_id], adjs)
          out = l3_emb.log_softmax(dim=-1)
          loss = F.nll_loss(out,y[n_id[:batch_size]])
          loss.backward()
          optimizer.step()

          total_loss += float(loss)
          total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())


      loss = total_loss / len(train_loader)


      approx_acc = total_correct / int(train_mask.sum())

      return loss, approx_acc
    #***************************************************
    optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

    for epoch in range(1,20):
      loss,acc = train(epoch)
      print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')
    #***************************************************

    CaliGraph_loader = NeighborSampler(data.edge_index,node_idx = cali_mask
    , sizes = [2,2,2],batch_size = 100000,shuffle=False)

    TestGraph_loader  = NeighborSampler(data.edge_index,node_idx = test_mask
    , sizes = [2,2,2],batch_size = 100000,shuffle=False)
    #***************************************************
    #function used to output final layer embedding and predicted labels
    def inference_1(subgraph_loader):
      model.eval()
      total_loss = total_correct = 0
      for batch_size,n_id,adjs in subgraph_loader:

          emb1,emb2,emb3 = model(x[n_id],adjs)
          out = emb3.log_softmax(dim=-1)
          y_pred = out.argmax(dim=-1,keepdim =True)
          c = x[n_id][:batch_size]
          fn = torch.cat((c, emb3.softmax(dim=-1)), dim=1)
          return fn,emb3, y_pred
    #***************************************************
    #Use this one

    fn_cali,emb3_Cali,y_pred_Cali = inference_1(CaliGraph_loader)
    emb3_Cali = emb3_Cali.softmax(dim=-1)
    numpy_emb3_Cali = emb3_Cali.detach().cpu().numpy()
    #print(numpy_emb3_Cali[0:10])
    #row_sums = np.sum(numpy_emb3_Cali, axis=1).reshape(-1, 1)
    #emb3_Cali = numpy_emb3_Cali/row_sums


    numpy_y_pred_Cali = y_pred_Cali.detach().cpu().numpy()
    numpy_y_labels_Cali = y[cali_mask].detach().cpu().numpy()
    numpy_y_labels_Test = y[test_mask].detach().cpu().numpy()
    numpy_y_labels_column = np.array(numpy_y_labels_Cali).reshape(-1, 1)
    #accuracy of our model for calibration nodes
    #acc = (numpy_y_pred_Cali == numpy_y_labels_column).sum()/cali_mask.sum()

    alpha = 0.05
    n=cali_mask.sum()
    m=test_mask.sum()

    #Using APS score function
    cal_smx_tensor = torch.FloatTensor(emb3_Cali)
    cal_smx = cal_smx_tensor.detach().cpu().numpy()
    cal_pi = cal_smx.argsort(1)[:, ::-1]
    cal_srt = np.take_along_axis(cal_smx,cal_pi,axis=1).cumsum(axis=1)
    cal_scores = np.take_along_axis(cal_srt, cal_pi.argsort(axis=1), axis=1)[
      range(n), numpy_y_labels_Cali
    ]

    qhat = np.quantile(
      cal_scores, np.ceil((n + 1) * (1 - alpha)) / n, interpolation="higher"
    )


    fn_test,emb3_Test,y_pred_Test = inference_1(TestGraph_loader)
    emb3_Test = emb3_Test.softmax(dim=-1)
    numpy_emb3_Test = emb3_Test.detach().cpu().numpy()
    test_smx = numpy_emb3_Test
    test_pi = test_smx.argsort(1)[:, ::-1]
    test_srt = np.take_along_axis(test_smx,test_pi,axis=1).cumsum(axis=1)

    test_scores = np.take_along_axis(test_srt, test_pi.argsort(axis=1), axis=1)[
      range(m), numpy_y_labels_Test
    ]


    prediction_sets = np.take_along_axis(test_srt <= qhat, test_pi.argsort(axis=1), axis=1)

    #***************************************************
    #print(prediction_sets)
    empirical = prediction_sets[
      np.arange(prediction_sets.shape[0]), numpy_y_labels_Test
    ].mean()

    print(f"The empirical marginal coverage is: {empirical}")
    #***************************************************
    #Now use CondConf to predict the interval
    # our conditional calibration method cover the test points
    #Important:we intentionally put the feature representation as the predicted probability to get the balanced coevrgae over every categories
    #To implement the alogorithm ,we should bear in mind the characteristics we care, in this case our goal is to reweight acoording to the predicted probabilities
    """
    #This auxiliary function aims to quickly calculate the covergae rate but not the specific prediction sets

    def computeCoverages(XCal, scoresCal, XTest, scoresTest, alpha):

      coveragesCond = np.zeros(len(XTest))
      for i in tqdm(range(len(XTest))):
          prob = setup_cvx_problem_calib(1-alpha,None,
                                                np.concatenate((scoresCal,np.array([scoresTest[i]]))), np.vstack((XCal,XTest[i,:])),{})
          prob.solve()
          #print(prob.constraints[2].shape)
          coveragesCond[i] = scoresTest[i] <= XTest[i,:]@prob.constraints[2].dual_value

      return coveragesCond

    coveragesCond = computeCoverages(numpy_emb3_Cali, cal_scores,numpy_emb3_Test, test_scores, alpha = 0.05)
    """

    #***************************************************
    #This cell defines fucntion that computes the ubs and lbs using the condconf by implementing the predicted probability feature represenation
    #fn has shape of [n_calib+n_test,data.num_features+1]
    #From Now the feature vector becomes the predicted probabilities
    x_caliandtest = np.vstack((numpy_emb3_Cali,numpy_emb3_Test))
    #x_caliandtest = np.hstack((x_caliandtest, np.ones((x_caliandtest.shape[0], 1))))
    fn = np.vstack((fn_cali.detach().cpu().numpy(),fn_test.detach().cpu().numpy()))
    num_features = data.num_features
    n_calib = int(cali_mask.sum())
    n_test = int(test_mask.sum())
    #matrix_1 represents class matrix

    num_classes = dataset.num_classes
    matrix_1 = np.zeros((n_calib+n_test, num_classes),dtype=int)

    for i in range(dataset.num_classes):
        matrix_1[:, i] = i

    caliandtest_srt = np.vstack((cal_srt, test_srt))
    caliandtest_pi = np.vstack((cal_pi,test_pi))

    #for each column we compute S(Xi,y) for all possible y categories(calibration and test)*****because we will use these in optimization problem

    test_scores_foreachclass=np.zeros((n_calib+n_test, dataset.num_classes))

    for i in range(dataset.num_classes):
        test_scores = (np.take_along_axis(caliandtest_srt , caliandtest_pi.argsort(axis=1), axis=1)[
        range(n_calib+n_test), matrix_1[:,i]])
        test_scores_foreachclass[:,i]=test_scores

    x_calib = x_caliandtest[:n_calib]
    y_calib = y[cali_mask].detach().cpu().numpy()
    x_test = x_caliandtest[-n_test:]
    y_test = y[test_mask].detach().cpu().numpy()
    alpha = 0.05
    #This function input packs of vectors and output the score of that x
    #I'll Rewrite it tomorrow for higher efficiency

    def score_fn(x_1,y):
      if(x_1.shape[1]!=1):
        #print('!=1')
        scores = np.ones((x_1.shape[0],1))
        for i in range(x_1.shape[0]):
          scores[i]=test_scores_foreachclass[i,y[i]]
        return scores.squeeze()
      else:
        #print('=1')
        for i in range(n_calib+n_test):
          if(np.array_equal(x_1,x_caliandtest[i])):
            return test_scores_foreachclass[i,y]

    def phi_n(x_2):

      if(len(x_2.shape)==2):

              return x_2
      else:

            return  x_2.reshape(1,dataset.num_classes)


    #given x_3 and s, output the prediction interval
    def score_inv_fn(s,x_3):

      for i in range(n_calib+n_test):

        if(np.array_equal(x_3.squeeze(),x_caliandtest[i])):

          return np.take_along_axis(caliandtest_srt[i].reshape(1,dataset.num_classes) <= s,caliandtest_pi[i].reshape(1,dataset.num_classes).argsort(axis=1),axis=1)
    def check_coverage(prediction_set):
          j=0
          for i in range(n_test):
            a = prediction_set[i].squeeze()
            if(a[y[test_mask][i]]== True):
              j=j+1
          return j/n_test



    infinite_params = {}
    cond_conf = CondConf(score_fn, phi_n, infinite_params)
    cond_conf.setup_problem(x_calib, y_calib)

    prediction_sets_test=[]
    for x_t in tqdm(x_test):

        res = cond_conf.predict(1-alpha, x_t, score_inv_fn)
        prediction_sets_test.append(res)
    print(f"The empirical marginal coverage using condition calibration is: {check_coverage(prediction_sets_test)}")

    prediction_sets_test = np.vstack(prediction_sets_test)
    def evaluate_predictions(S, X, y):
        marg_coverage = np.mean([y[i] in S[i] for i in range(len(y))])
        wsc_coverage = coverage.wsc_unbiased(X, y, S)
        print('Marginal coverage:       {:2.3%}'.format(marg_coverage))
        print('WS conditional coverage: {:2.3%}'.format(wsc_coverage))
    def indices_of_nonzero_entries(binary_matrix):
        # Using list comprehension to find the indices of non-zero entries for each row
        indices_matrix = [np.flatnonzero(row) for row in binary_matrix]
        return indices_matrix

    int_matrix =prediction_sets.astype(int)
    indices_matrix = indices_of_nonzero_entries(int_matrix)
    print(f"The empirical marginal and WS conditional coverage using split conformal is: ")
    print(evaluate_predictions(indices_matrix,original_x[test_mask].detach().cpu().numpy(),y[test_mask].detach().cpu().numpy()))
    int_matrix =prediction_sets_test.astype(int)
    indices_matrix = indices_of_nonzero_entries(int_matrix)
    print(f"The empirical marginal and WS conditional coverage using conditional calibration is: ")
    print(evaluate_predictions(indices_matrix,original_x[test_mask].detach().cpu().numpy(),y[test_mask].detach().cpu().numpy()))

    marginal_split.append(empirical)
    marginal_split.append(check_coverage(prediction_sets_test))
    prediction_split.append(prediction_sets)
    prediction_condi.append(prediction_sets_test)
    test_mask_list.append(test_mask.numpy().astype(int))


In [ ]:
print(empirical)
print(check_coverage(prediction_sets_test))
print(prediction_sets_test)

In [ ]:
#Store our data
def serialize_ndarrays(list_of_ndarrays):
    return '\n\n'.join('\n'.join(' '.join(str(cell) for cell in row) for row in arr) if arr.ndim > 1 else ' '.join(str(cell) for cell in arr) for arr in list_of_ndarrays)



list_of_numpy_arrays_train = [tensor.numpy() for tensor in train_index if tensor.requires_grad == False]
list_of_numpy_arrays_calib = [tensor.numpy() for tensor in calib_index if tensor.requires_grad == False]
list_of_numpy_arrays_test = [tensor.numpy() for tensor in test_index if tensor.requires_grad == False]
test_index_int = [arr.astype(int) for arr in list_of_numpy_arrays_test]
test_index_string = serialize_ndarrays(test_index)
prediction_set_string =serialize_ndarrays(prediction_sets)
prediction_condi_string = serialize_ndarrays(prediction_condi)
marginal_condi_string = serialize_ndarrays(marginal_condi)
marginal_split_string = serialize_ndarrays(marginal_split)

file_path = '/content/drive/My Drive/Classification_Chicago_10runs.txt'
separator = "\n" + "-" * 50 + "\n"  # Creates a line of dashes as a separator.

with open(file_path, 'w') as f:
     f.write(test_index_string + separator)
     #f.write(calib_index_string + separator)
    # f.write(test_index_string + separator)
   # f.write(marginal_condi_string + separator)
   # f.write(marginal_split_string + separator)
     f.write(prediction_set_string + separator)
     f.write(prediction_condi_string)

